### Overview of the dataset:
The attached dataset is a baseline – aggregated data holding key performance metrics across different selling points for different products. Each product can be represented in different selling points (DistributionUnit) and have different prices depending on where is being sold.

### Explanation of some fields:


BaselineDetailID – Id of the product in the DistributionUnit

BaselineID – ID of current baseline.

ProductID – General ID of the Product

DistributionUnit – sales channel.

NSRperUC -  Net Sales Revenue Per Unit Case, presented only for TCCC Manufacturer.

COGS – Cost of goods sold

ExitRate – Percentage of the volume that will be lost if the product is not represented in the store

### Overview of the task:
Create an executive summary for a provided baseline that can be shown to a market owner to help him better understand the market structure and TCCC's position. Below is the list of the visualizations that might be helpful, but this list is not a hard restriction.

### High-level aggregations:

- top Manufacturers

- top Brands by Volume and Manufacturer

- top Categories by Volume

- top Brand PackSize by Channel and Manufacturer by Volume for all, by Revenue for TCCC

### Competitor selection:

- for the top 3 TCCC brands: the top 3 SKUs from each brand present the most suitable competitor SKU.
Describe what criteria are used to define competitor SKU and why. How do TCCC SKUs perform in contrast to competitor SKUs?

### Product Performance:

- what are the key products that must always be presented in the store to avoid loss of sales? (Hist Exit Rates)
- what are the key products for which price change can lead to an unprofitable loss of sales? (Hint: Elasticity)

The result should be presented in a jupyter-notebook with clear, interactive visualizations. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('products-metrics/DemoBaseline.xlsx')
df.head()

,BaselineDetailID,BaselineID,ProductID,DistributionUnit,CurrencyCode,Price,NSRperUC,EnvironmentalTaxPerUC,OperatingCOGSperUC,WarehouseCostPerUC,...,ExitRate,RatioNonOperatingCOGStoNSR,Distribution,Description,Manufacturer,Category,Brand,Flavor,PackSize,Packaging
0,816018,451,358130194,SOUTH,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
1,816019,451,358130194,EAST,484,65.9368,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
2,816020,451,358130194,WEST,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,9.00000,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
3,816021,451,358130194,NORTH,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
4,816014,451,358130195,SOUTH,484,126.7034,75.13333,0,22.429,0,...,9.63984,0,0.12124,ADAS_COCA-COLA COMPANY_DEL VALLE Y NADA_NARANJ...,TCCC,ADAS,DEL VALLE Y NADA,NARANJA,600,BOTELLA DE PLASTICO_NR


In [3]:
df.shape

(2843, 24)

In [4]:
df.columns

Index(['BaselineDetailID', 'BaselineID', 'ProductID', 'DistributionUnit',
       'CurrencyCode', 'Price', 'NSRperUC', 'EnvironmentalTaxPerUC',
       'OperatingCOGSperUC', 'WarehouseCostPerUC', 'DistributionCostPerUC',
       'TransactionsPerUC', 'Volume', 'PriceElasticity', 'ExitRate',
       'RatioNonOperatingCOGStoNSR', 'Distribution', 'Description',
       'Manufacturer', 'Category', 'Brand', 'Flavor', 'PackSize', 'Packaging'],
      dtype='object')

# High-level aggregations:

### - top Manufacturers


In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [6]:
df.groupby(['Manufacturer']).agg({'Volume': 'sum'})

,Volume
Manufacturer,
AGA DE MEXICO,20749670.020
AGGREGATED,2859033.851
AJEMEX,1298721.620
ARIZONA,1149579.784
BELTICOS,12859.343
BLANQUITA,1585048.256
CALAHUA,199.736
CAMPBELL'S,62405.636
DANONE,12183420.037


### - top Brands by Volume and Manufacturer


In [7]:
brand_manuf = df.groupby(['Manufacturer', 'Brand']) \
                .agg({'Volume': 'sum'}) \
                .sort_values(by='Volume', ascending=False)
brand_manuf.head()

,,Volume
Manufacturer,Brand,
TCCC,COCA COLA,1041345057.730
PEPSICO,PEPSI COLA,43262799.577
TCCC,CIEL,27188346.298
GRUPO GEPP,E-PURA,21817687.247
DANONE,BONAFONT,12183420.037



### - top Categories by Volume


In [8]:
categ = df.groupby('Category') \
          .agg({'Volume': 'sum'}) \
          .sort_values(by='Volume', ascending=False)
categ.head()

,Volume
Category,
COLAS,1107280612.749
AGUA EMBOTELLADA,65803324.272
R. FRUTALES,62894655.679
NARANJADAS,13490668.048
AGUA MINERAL NATURAL,7764271.725


### - top Brand PackSize by Channel and Manufacturer by Volume for all, by Revenue for TCCC

In [9]:
# add revenue column
df['Revenue'] = df['Price'] * df['Volume']
# define DataFrame of TCCC
tccc_df = df[df['Manufacturer'] == "TCCC"]

In [11]:
# group the dataframe by PackSize, Brand, Channel and Manufacturer and calculate the sum of Volume
packsizes = df.groupby(['PackSize', 'Brand', 'DistributionUnit', 'Manufacturer']) \
              .agg({'Volume': 'sum'}) \
              .reset_index()

In [12]:
# get top PackSizes by Volume
top_ps = packsizes.groupby(['Brand', 'DistributionUnit', 'Manufacturer']) \
                  .agg({'Volume': 'max'}) \
                  .reset_index()

In [13]:
# join tables to get PackSize
pack_size = top_ps.merge(packsizes, on='Volume', suffixes=('_', '')).loc[:, 'Volume':'Manufacturer']
pack_size

,Volume,PackSize,Brand,DistributionUnit,Manufacturer
0,5620.389,1000,ACAPULCO,NORTH,LA CONCORDIA
1,2427.024,1000,ACAPULCO,SOUTH,LA CONCORDIA
2,3034.808,1000,ACAPULCO,WEST,LA CONCORDIA
3,113.400,330,ACAPULCOCO,NORTH,CALAHUA
4,25.110,330,ACAPULCOCO,SOUTH,CALAHUA
...,...,...,...,...,...
485,66.180,3785,ZAMBA,EAST,DISTRIB. DE PRODS. ALIMEN.
486,25.104,600,ZUBBA,EAST,AGA DE MEXICO
487,58346.601,2000,ZUBBA,NORTH,AGA DE MEXICO
488,31833.380,2000,ZUBBA,SOUTH,AGA DE MEXICO


### TCCC - top brands by Revenue

In [14]:
# group the TCCC dataframe by PackSize, Brand, Channel and calculate the sum of Revenue
tccc_packsizes = df.groupby(['PackSize', 'Brand', 'DistributionUnit']) \
                   .agg({'Revenue': 'sum'}) \
                   .reset_index() 

In [15]:
# get top TCCC PackSizes by Revenue
tccc_top_ps = tccc_packsizes.groupby(['Brand', 'DistributionUnit']) \
                            .agg({'Revenue': 'max'}) \
                            .reset_index()

In [16]:
# join TCCC tables to get PackSize
tccc_pack_size = tccc_top_ps.merge(tccc_packsizes, on='Revenue', suffixes=('_', '')).loc[:, 'Revenue':'DistributionUnit']
tccc_pack_size

,Revenue,PackSize,Brand,DistributionUnit
0,204018.423,1000,ACAPULCO,NORTH
1,88100.251,1000,ACAPULCO,SOUTH
2,110162.619,1000,ACAPULCO,WEST
3,32831.974,330,ACAPULCOCO,NORTH
4,7269.903,330,ACAPULCOCO,SOUTH
...,...,...,...,...
457,2049.973,3785,ZAMBA,EAST
458,2825.169,600,ZUBBA,EAST
459,4991195.760,600,ZUBBA,NORTH
460,2723151.481,600,ZUBBA,SOUTH


### - Top Manufacturer by Volume

In [17]:
manuf = df.groupby(['Manufacturer']) \
          .agg({'Volume': 'sum'}) \
          .sort_values(by='Volume', ascending=False)
manuf.head()

,Volume
Manufacturer,
TCCC,1119014111.135
PEPSICO,60924697.840
GRUPO GEPP,25039622.726
AGA DE MEXICO,20749670.020
DANONE,12183420.037


# Competitor selection:

- for the top 3 TCCC brands: the top 3 SKUs from each brand present the most suitable competitor SKU.
Describe what criteria are used to define competitor SKU and why. How do TCCC SKUs perform in contrast to competitor SKUs?

## Top 3 TCCC brands 
Brands are chosen by Volume

In [18]:
tccc_top = df[df['Manufacturer'] == 'TCCC'].groupby('Brand') \
            .agg({'Volume': 'sum'}) \
            .sort_values(by='Volume', ascending=False) \
            .reset_index() \
            .head(3)

In [19]:
tccc_top

,Brand,Volume
0,COCA COLA,1041345057.730
1,CIEL,27188346.298
2,SPRITE,6284246.316


## Top products for each top brand of TCCC
Products are also taken by Volume

In [21]:
tccc_top_products = {}  # dict with top TCCC brands : dataframe with top products by Volume
for brand in tccc_top['Brand']:
    tccc_top_products[brand] = df[df['Brand'] == brand] \
                                .groupby(['ProductID', 'Category']) \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)

In [22]:
tccc_top_products['COCA COLA']

,ProductID,Category,Volume
0,358147329,COLAS,633709730.893
1,358130403,COLAS,42362988.933
2,358130405,COLAS,23246014.955


In [23]:
tccc_top_products['CIEL']

,ProductID,Category,Volume
0,358130211,AGUA EMBOTELLADA,14720499.468
1,358130213,AGUA EMBOTELLADA,4846272.425
2,358130215,AGUA EMBOTELLADA,3984153.997


In [24]:
tccc_top_products['SPRITE']

,ProductID,Category,Volume
0,358130586,R. FRUTALES,1597090.821
1,358130587,R. FRUTALES,1258110.077
2,358130590,R. FRUTALES,975360.892


### DataFrame of TCCC competitors

In [25]:
no_tccc_df = df[(df['Manufacturer'] != 'TCCC')]
no_tccc_df.head(3)

,BaselineDetailID,BaselineID,ProductID,DistributionUnit,CurrencyCode,Price,NSRperUC,EnvironmentalTaxPerUC,OperatingCOGSperUC,WarehouseCostPerUC,...,RatioNonOperatingCOGStoNSR,Distribution,Description,Manufacturer,Category,Brand,Flavor,PackSize,Packaging,Revenue
0,816018,451,358130194,SOUTH,484,62.876,0.000,0,0.000,0,...,0,0.001,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,4334209.672
1,816019,451,358130194,EAST,484,65.937,0.000,0,0.000,0,...,0,0.001,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,702789.722
2,816020,451,358130194,WEST,484,62.876,0.000,0,0.000,0,...,0,9.000,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,1000620.198


## Top competitors for COCA COLA SKUs
Since all COCA COLA (CC) products are in the same category, the competitors' products are the same for each CC product

In [26]:
tccc_top_products['COCA COLA']

,ProductID,Category,Volume
0,358147329,COLAS,633709730.893
1,358130403,COLAS,42362988.933
2,358130405,COLAS,23246014.955


In [27]:
# get products that are in the same category that CC SKUs, and get top-3 of them by Volume
cc_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'COCA COLA') & (no_tccc_df['Category'] == 'COLAS')] \
                            .groupby('ProductID') \
                            .agg({'Volume': 'sum'}) \
                            .sort_values(by='Volume', ascending=False) \
                            .reset_index() \
                            .head(3)
cc_competitors

,ProductID,Volume
0,358130417,14586191.625
1,358130420,11730571.401
2,358130416,7835953.230


In [28]:
# list of COCA COLA competitor brands
cc_comp_mask = (no_tccc_df['ProductID'].isin(cc_competitors['ProductID']))
no_tccc_df[cc_comp_mask]['Brand'].unique()

array(['PEPSI COLA'], dtype=object)

As we see, the only competitor to COCA COLA products is PEPSI COLA brand. 
<br>The volume of PEPSI COLA products is much (~ 1.5-40 times) less than the volume of COCA COLA products.

## Top competitors for CIEL SKUs
Since all CIEL products are in the same category, the competitors' products are the same for each CIEL product

In [29]:
tccc_top_products['CIEL']

,ProductID,Category,Volume
0,358130211,AGUA EMBOTELLADA,14720499.468
1,358130213,AGUA EMBOTELLADA,4846272.425
2,358130215,AGUA EMBOTELLADA,3984153.997


In [30]:
# get products that are in the same category that CIEL SKUs, and get top-3 of them by Volume
ciel_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'CIEL') 
                               & (no_tccc_df['Category'] == 'AGUA EMBOTELLADA')].groupby('ProductID') \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)
ciel_competitors

,ProductID,Volume
0,358130212,14064426.918
1,358130214,5361399.371
2,358130220,2514825.811


In [31]:
# list of CIEL competitor brands
ciel_comp_mask = (no_tccc_df['ProductID'].isin(ciel_competitors['ProductID']))
no_tccc_df[ciel_comp_mask]['Brand'].unique()

array(['E-PURA', 'BONAFONT'], dtype=object)

CIEL competitors are E-PURA and BONAFONT.
<br>CIEL competitors are almost on the same level based on Volume metric.

## Top competitors for SPRITE SKUs
Since all SPRITE products are in the same category, the competitors' products are the same for each SPRITE product

In [32]:
tccc_top_products['SPRITE']

,ProductID,Category,Volume
0,358130586,R. FRUTALES,1597090.821
1,358130587,R. FRUTALES,1258110.077
2,358130590,R. FRUTALES,975360.892


In [33]:
# get products that are in the same category that SPRITE SKUs, and get top-3 of them by Volume
sprite_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'SPRITE') 
                               & (no_tccc_df['Category'] == 'R. FRUTALES')].groupby('ProductID') \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)
sprite_competitors

,ProductID,Volume
0,358130584,7704361.056
1,358130593,2742334.943
2,358130707,2415774.413


In [34]:
# list of SPRITE competitor brands
sprite_comp_mask = (no_tccc_df['ProductID'].isin(sprite_competitors['ProductID']))
no_tccc_df[sprite_comp_mask]['Brand'].unique()

array(['SQUIRT', 'SEVEN UP', 'AGA'], dtype=object)

In [35]:
no_tccc_df[no_tccc_df['ProductID'] == 358130584]['Brand'].unique()

array(['SQUIRT'], dtype=object)

SPRITE competitors are SQUIRT, SEVEN UP, and AGA.
<br>SPRITE competitors are better than SPRITE based on Volume metric. The top Volume of SQUIRT is 4.8 times greater than the top Volume of SPRITE

# Product Performance:

- what are the key products that must always be presented in the store to avoid loss of sales? (Hist Exit Rates)


In [36]:
# add the column of the absolute amount of the lost volume
df['LostVolume'] = df['Volume'] * df['ExitRate']

In [37]:
# get top-5 products with brands with the biggest volume loss
df.groupby(['ProductID', 'Brand']) \
  .agg({'LostVolume': 'sum'}) \
  .sort_values('LostVolume', ascending=False) \
  .head()

,,LostVolume
ProductID,Brand,
358147329,COCA COLA,9093734638.308
358130403,COCA COLA,598256060.530
358130405,COCA COLA,285495002.829
358146312,COCA COLA,264040000.000
358146307,COCA COLA,264040000.000



- what are the key products for which price change can lead to an unprofitable loss of sales? (Hint: Elasticity)

In [38]:
# add the column of the loss of revenue
df['RevenueLoss'] = df['Revenue'] * df['PriceElasticity']

In [39]:
df.groupby(['ProductID', 'Brand']) \
  .agg({'RevenueLoss':'sum'}) \
  .sort_values('RevenueLoss') \
  .head()

,,RevenueLoss
ProductID,Brand,
358147329,COCA COLA,-80531832601.821
358147326,COCA COLA,-3588000000.000
358130403,COCA COLA,-2487830042.574
358146312,COCA COLA,-2351520000.000
358146307,COCA COLA,-2351520000.000
